In [10]:
import igraph as ig
import itertools
from statistics import mean
from sklearn.preprocessing import minmax_scale


In [11]:
period = 'april2022'
full_graph_path = f'../../data/graphs/{period}.graphml'
g = ig.Graph.Read(full_graph_path)


In [12]:
g_L_space = ig.Graph(directed=True)


In [13]:
# Adding all vertex from original graph
g_L_space.add_vertices(
    n=g.vs['name'],
    attributes={
        'label': g.vs['label'],
        'lat': g.vs['lat'],
        'lon': g.vs['lon']
    }
)


In [14]:
for a, b in itertools.permutations(g.vs, 2):
    edges = g.es.select(_source=a.index, _target=b.index)
    if len(edges) > 0:
        g_L_space.add_edge(
            source=a.index,
            target=b.index,
            time_second=mean([e['time_second'] for e in edges]),
            num_train=sum([e['num_train'] for e in edges])
        )


In [15]:
g_L_space.vs['total_train'] = [
    max(
        sum(edge['num_train']
            for edge in g_L_space.es.select(_source=vertex.index)),
        sum(edge['num_train']
            for edge in g_L_space.es.select(_target=vertex.index))
    )
    for vertex in g_L_space.vs
]


In [16]:
g_L_space.summary()


'IGRAPH DN-- 428 1173 -- \n+ attr: label (v), lat (v), lon (v), name (v), total_train (v), num_train (e), time_second (e)'

In [17]:
g_L_space.vs['degree'] = minmax_scale(g_L_space.degree())
g_L_space.vs['betweennes'] = minmax_scale(g_L_space.betweenness(directed=True))
g_L_space.vs['closeness'] = minmax_scale(g_L_space.closeness())
g_L_space.vs['pagerank'] = minmax_scale(g_L_space.pagerank())
g_L_space.vs['strength_num_train'] = minmax_scale(g_L_space.strength(weights='num_train'))
g_L_space.vs['pagerank_num_train'] = minmax_scale(g_L_space.pagerank(weights='num_train'))

In [18]:
# Save the graph
save_path = f'../../data/graphs/{period}_Lspace.graphml'
g_L_space.write_graphml(save_path)
